In [1]:
import operator
import math
import random
import matplotlib.pyplot as plt

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

C:\Users\Péricles\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

In [3]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [4]:
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

In [5]:
toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [6]:
def main():
    random.seed(318)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 30, stats=mstats,
                                   halloffame=hof, verbose=True)
    
    return pop, log, hof

In [7]:
def plot(original_function, candidate_function):
        
        x_values = [x/10. for x in range(-10,10)]
        
        print(x_values)
        
        original_points = []
        candidate_points = []
        
        for value in x_values:
            original_points.append(original_function(value))
            candidate_points.append(candidate_function(value))
        
        plt.plot(x_values, original_points, 'k', x_values, candidate_points, 'r')
        plt.show()

In [9]:
if __name__ == "__main__":
    results = main()
    
    expr = results[2][0]
    
    print(expr)
    
    
    '''PLOTANDO OS GRAFICOS
    tree = gp.PrimitiveTree(expr)
    
    candidate_function = toolbox.compile(tree)
    

    original_function = lambda x: x**4 + x**3 + x**2 + x
    
    
    plot(original_function, candidate_function)
    '''
    

   	      	                fitness                	              size             
   	      	---------------------------------------	-------------------------------
gen	nevals	avg   	max  	min     	std    	avg 	max	min	std    
0  	50    	2.0373	30.34	0.483609	4.16881	3.82	7  	2  	1.76284
1  	29    	1.11548	3.35292	0.483609	0.633998	3.2 	7  	2  	1.29615
2  	28    	1.09026	2.9829 	0.564583	0.626726	3.18	7  	1  	1.32197
3  	35    	0.82779	2.9829 	0.564583	0.440574	3.08	9  	1  	1.38333
4  	32    	0.745502	2.552  	0.564583	0.351867	3.32	9  	1  	2.01435
5  	32    	0.742672	2.3179 	0.328818	0.483746	3.18	9  	1  	2.18806
6  	29    	0.652508	1.81191	0.328818	0.251397	2.98	9  	1  	2.06388
7  	31    	0.880678	2.9829 	0.328818	0.726034	3.38	9  	1  	2.30556
8  	33    	0.62051 	2.3179 	0.328818	0.332198	3.98	14 	1  	2.90165
9  	20    	0.553641	2.54124	0.328818	0.359949	4.68	13 	1  	2.97617
10 	31    	0.687706	2.54124	0.328818	0.609318	5.52	11 	1  	2.34299
11 	27    	0.47376 	2.76958	0.328818	0.3960